In [2]:
from __future__ import annotations
from typing import Literal
from pydantic import BaseModel, field_validator, model_validator


class Course(BaseModel):
    course_name: str
    description: str
    difficulty: str  # validated below
    prerequisites: list[str]

    @field_validator("difficulty")
    @classmethod
    def validate_difficulty(cls, v: str) -> str:
        allowed = {"Beginner", "Intermediate", "Advanced"}
        if v not in allowed:
            raise ValueError(f"difficulty must be one of {allowed}, got '{v}'")
        return v


class CoursePlan(BaseModel):
    is_broad: bool
    total_courses: int
    courses: list[Course]

    @model_validator(mode="after")
    def validate_plan(self) -> "CoursePlan":
        if len(self.courses) > 8:
            raise ValueError(f"Maximum 8 courses allowed, got {len(self.courses)}")
        # Auto-correct total_courses if the LLM miscounts
        if self.total_courses != len(self.courses):
            object.__setattr__(self, "total_courses", len(self.courses))
        return self


print("✅ Pydantic models defined.")

✅ Pydantic models defined.


## 4. `plan_courses()` — Public API

In [4]:
import json

def plan_courses(course_title: str, course_description: str) -> CoursePlan:
    """
    Analyze a course topic and return a validated CoursePlan.

    Args:
        course_title       : Title of the course or topic.
        course_description : Brief description of what the course covers.

    Returns:
        A validated CoursePlan instance.
    """
    return chain.invoke({
        "course_title": course_title,
        "course_description": course_description,
    })


def display_plan(plan: CoursePlan) -> None:
    """Pretty-print a CoursePlan to the notebook output."""
    badge = {"Beginner": "🟢", "Intermediate": "🟡", "Advanced": "🔴"}
    print(f"Topic type    : {'🌐 Broad' if plan.is_broad else '🎯 Narrow'}")
    print(f"Total courses : {plan.total_courses}")
    print("-" * 60)
    for i, c in enumerate(plan.courses, 1):
        print(f"  {i}. {badge.get(c.difficulty, '⚪')} [{c.difficulty}] {c.course_name}")
        print(f"     {c.description}")
        if c.prerequisites:
            print(f"     Prerequisites: {', '.join(c.prerequisites)}")
        print()

print("✅ plan_courses() ready.")

✅ plan_courses() ready.


In [5]:
narrow_plan = plan_courses(
    course_title="Python List Comprehensions",
    course_description=(
        "A focused guide on writing concise and efficient list comprehensions "
        "in Python, including filtering, nested comprehensions, and performance tips."
    ),
)

display_plan(narrow_plan)

# ── Assertions ─────────────────────────────────────────────────────────────
assert narrow_plan.is_broad == False,                          "Expected is_broad=False"
assert narrow_plan.total_courses == len(narrow_plan.courses),  "total_courses mismatch"
assert len(narrow_plan.courses) <= 8,                          "Too many courses"
print("✅ All assertions passed (narrow topic).")

Topic type    : 🎯 Narrow
Total courses : 1
------------------------------------------------------------
  1. 🟢 [Beginner] Python List Comprehensions Basics
     Learn the foundational syntax and usage of list comprehensions in Python for simple tasks.

✅ All assertions passed (narrow topic).


In [6]:
broad_plan = plan_courses(
    course_title="Machine Learning",
    course_description=(
        "A comprehensive learning path covering all aspects of machine learning, "
        "from foundational mathematics and Python to supervised learning, "
        "unsupervised learning, deep learning, and deploying ML models in production."
    ),
)

display_plan(broad_plan)

# ── Assertions ─────────────────────────────────────────────────────────────
assert broad_plan.is_broad == True,                          "Expected is_broad=True"
assert broad_plan.total_courses == len(broad_plan.courses),  "total_courses mismatch"
assert 2 <= len(broad_plan.courses) <= 8,                    f"Expected 2-8 courses, got {len(broad_plan.courses)}"
valid = {"Beginner", "Intermediate", "Advanced"}
for c in broad_plan.courses:
    assert c.difficulty in valid, f"Invalid difficulty: {c.difficulty}"
print("✅ All assertions passed (broad topic).")

Topic type    : 🌐 Broad
Total courses : 8
------------------------------------------------------------
  1. 🟢 [Beginner] Foundations of Machine Learning
     Introduction to the mathematical and statistical foundations necessary for machine learning.

  2. 🟢 [Beginner] Python Programming Basics
     Learn Python syntax, data structures, control flow and basic programming concepts essential for machine learning.
     Prerequisites: Foundations of Machine Learning

  3. 🟡 [Intermediate] Data Preprocessing Techniques
     Understand how to clean, normalize and transform data for effective machine learning.
     Prerequisites: Python Programming Basics

  4. 🟡 [Intermediate] Supervised Learning Algorithms
     Explore various supervised learning algorithms and their applications in real-world problems.
     Prerequisites: Data Preprocessing Techniques

  5. 🟡 [Intermediate] Unsupervised Learning Algorithms
     Learn about clustering, dimensionality reduction and other unsupervised learnin

In [7]:
# ── Enter your topic here ──────────────────────────────────────────────────
my_title       = "Web Development with React"
my_description = (
    "Learn to build modern, responsive web applications using React, "
    "covering components, hooks, state management, routing, and API integration."
)
# ──────────────────────────────────────────────────────────────────────────

my_plan = plan_courses(my_title, my_description)
display_plan(my_plan)

Topic type    : 🌐 Broad
Total courses : 4
------------------------------------------------------------
  1. 🟢 [Beginner] React Fundamentals
     Introduction to React basics and JSX syntax.

  2. 🟡 [Intermediate] Advanced React Components
     Deep dive into advanced component patterns like higher-order components, render props, and context API.
     Prerequisites: React Fundamentals

  3. 🟡 [Intermediate] State Management in React Apps
     Explore Redux for state management and learn to implement global states with Context API.
     Prerequisites: React Fundamentals

  4. 🔴 [Advanced] Building Full-Stack Apps with React & Node.js
     Create a complete web application using both frontend and backend technologies.
     Prerequisites: React Fundamentals, State Management in React Apps



## 8. Raw JSON Output

In [8]:
# View the validated JSON for the last plan
print(json.dumps(my_plan.model_dump(), indent=2))

{
  "is_broad": true,
  "total_courses": 4,
  "courses": [
    {
      "course_name": "React Fundamentals",
      "description": "Introduction to React basics and JSX syntax.",
      "difficulty": "Beginner",
      "prerequisites": []
    },
    {
      "course_name": "Advanced React Components",
      "description": "Deep dive into advanced component patterns like higher-order components, render props, and context API.",
      "difficulty": "Intermediate",
      "prerequisites": [
        "React Fundamentals"
      ]
    },
    {
      "course_name": "State Management in React Apps",
      "description": "Explore Redux for state management and learn to implement global states with Context API.",
      "difficulty": "Intermediate",
      "prerequisites": [
        "React Fundamentals"
      ]
    },
    {
      "course_name": "Building Full-Stack Apps with React & Node.js",
      "description": "Create a complete web application using both frontend and backend technologies.",
      "di